<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Quant_Analyst_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dask

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import dask.dataframe as dd
import pandas as pd
from sqlalchemy import create_engine

sql_engine = create_engine("sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db")
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"
tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''

tables_df = pd.read_sql_query(tables_query, sql_engine)

tables_ddf = dd.from_pandas(tables_df, npartitions=1)
relevant_table_names = tables_ddf[tables_ddf['name'].str[-4:].astype(int) > 2017]['name'].compute().tolist()

count = len(relevant_table_names)

print("Relevant Table Names:")
print(relevant_table_names)
print("Count of trading days from 2018 onwards:", count)


Relevant Table Names:
['01012018', '02012018', '03012018', '04012018', '05012018', '08012018', '09012018', '10012018', '11012018', '12012018', '15012018', '16012018', '17012018', '18012018', '19012018', '22012018', '23012018', '24012018', '25012018', '29012018', '30012018', '31012018', '01022018', '02022018', '05022018', '06022018', '07022018', '08022018', '09022018', '12022018', '14022018', '15022018', '16022018', '19022018', '20022018', '21022018', '22022018', '23022018', '26022018', '27022018', '28022018', '01032018', '05032018', '06032018', '07032018', '08032018', '09032018', '12032018', '13032018', '14032018', '15032018', '16032018', '19032018', '20032018', '21032018', '22032018', '23032018', '26032018', '27032018', '28032018', '02042018', '03042018', '04042018', '05042018', '06042018', '09042018', '10042018', '11042018', '12042018', '13042018', '16042018', '17042018', '18042018', '19042018', '20042018', '23042018', '24042018', '25042018', '26042018', '27042018', '30042018', '0205

In [12]:
import dask.dataframe as dd
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Create a select statement
    select_stmt = select([table]).alias()
    # Read data into Dask DataFrame
    df_dask = dd.read_sql_table(select_stmt, connection_string, index_col='time', npartitions=4)
    dfs.append(df_dask)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Print the head of the merged DataFrame
print(merged_df.head())


ArgumentError: Column expression, FROM clause, or other columns clause element expected, got [Table('01012018', MetaData(), Column('time', TEXT(), table=<01012018>), Column('symbol', TEXT(), table=<01012018>), Column('open', REAL(), table=<01012018>), Column('high', REAL(), table=<01012018>), Column('low', REAL(), table=<01012018>), Column('close', REAL(), table=<01012018>), Column('volume', INTEGER(), table=<01012018>), Column('oi', INTEGER(), table=<01012018>), Column('expiry', TEXT(), table=<01012018>), Column('strike', INTEGER(), table=<01012018>), Column('instrument_type', TEXT(), table=<01012018>), schema=None)]. Did you mean to say select(Table('01012018', MetaData(), Column('time', TEXT(), table=<01012018>), Column('symbol', TEXT(), table=<01012018>), Column('open', REAL(), table=<01012018>), Column('high', REAL(), table=<01012018>), Column('low', REAL(), table=<01012018>), Column('close', REAL(), table=<01012018>), Column('volume', INTEGER(), table=<01012018>), Column('oi', IN

In [14]:
import dask.dataframe as dd
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table).alias()
    # Read data into Dask DataFrame
    df_dask = dd.read_sql_table(select_stmt, connection_string, index_col='time', npartitions=4)
    dfs.append(df_dask)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Print the head of the merged DataFrame
print(merged_df.head())


TypeError: `table_name` must be of type str, not <class 'sqlalchemy.sql.selectable.Subquery'>

In [16]:
import dask.dataframe as dd
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Convert result to DataFrame
        df = dd.from_delayed([result.fetchall()], meta=result._metadata)
        dfs.append(df)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Set the index column
merged_df = merged_df.set_index('time')

# Print the head of the merged DataFrame
print(merged_df.head())



TypeError: Expected Delayed object, got list

In [17]:
import dask.dataframe as dd
from dask.delayed import delayed
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Convert result to delayed object
        delayed_result = delayed(result.fetchall)()
        # Convert delayed result to DataFrame
        df = dd.from_delayed([delayed_result], meta=result._metadata)
        dfs.append(df)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Set the index column
merged_df = merged_df.set_index('time')

# Print the head of the merged DataFrame
print(merged_df.head())


TypeError: Don't know how to create metadata from <sqlalchemy.engine.cursor.CursorResultMetaData object at 0x7c94c9059700>

In [18]:
import dask.dataframe as dd
from dask.delayed import delayed
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Convert result to delayed object
        delayed_result = delayed(result.fetchall)()
        # Create Dask metadata manually
        meta = {
            'time': str(result._metadata.columns['time'].type),
            'symbol': str(result._metadata.columns['symbol'].type),
            'open': str(result._metadata.columns['open'].type),
            'high': str(result._metadata.columns['high'].type),
            'low': str(result._metadata.columns['low'].type),
            'close': str(result._metadata.columns['close'].type),
            'volume': str(result._metadata.columns['volume'].type),
            'oi': str(result._metadata.columns['oi'].type),
            'expiry': str(result._metadata.columns['expiry'].type),
            'strike': str(result._metadata.columns['strike'].type),
            'instrument_type': str(result._metadata.columns['instrument_type'].type)
        }
        # Convert delayed result to DataFrame
        df = dd.from_delayed([delayed_result], meta=meta)
        dfs.append(df)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Set the index column
merged_df = merged_df.set_index('time')

# Print the head of the merged DataFrame
print(merged_df.head())


AttributeError: 'CursorResultMetaData' object has no attribute 'columns'

In [22]:
import dask.dataframe as dd
from dask.delayed import delayed
from sqlalchemy import create_engine, MetaData, Table, select

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Read data from each table using Dask
dfs = []
for table_name in relevant_table_names:
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Convert result to delayed object
        delayed_result = delayed(result.fetchall)()
        # Create Dask metadata manually
        meta = {
            'time': 'datetime64[ns]',  # Assuming 'time' column is of datetime type
            'symbol': str,  # Assuming 'symbol' column is of string type
            'open': float,  # Assuming 'open' column is of float type
            'high': float,  # Assuming 'high' column is of float type
            'low': float,   # Assuming 'low' column is of float type
            'close': float, # Assuming 'close' column is of float type
            'volume': int,  # Assuming 'volume' column is of integer type
            'oi': int,      # Assuming 'oi' column is of integer type
            'expiry': str,  # Assuming 'expiry' column is of string type
            'strike': int,  # Assuming 'strike' column is of integer type
            'instrument_type': str  # Assuming 'instrument_type' column is of string type
        }
        # Convert delayed result to DataFrame
        df = dd.from_delayed([delayed_result], meta=meta)
        dfs.append(df)

# Concatenate the dataframes
merged_df = dd.concat(dfs)

# Set the index column
merged_df = merged_df.set_index('time')

# Print the head of the merged DataFrame
print(merged_df.head())


ValueError: Metadata mismatch found in `from_delayed`.

Expected partition of type `pandas.core.frame.DataFrame` but got `list`

In [ ]:
import dask.dataframe as dd
from sqlalchemy import create_engine, MetaData, Table, select
from dask.delayed import delayed

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Function to read each table as a Dask DataFrame
def read_table(table_name):
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement and fetch results into a Dask DataFrame
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Convert each row to a delayed object
        delayed_results = [delayed(row) for row in result.fetchall()]
        # Create Dask metadata manually
        meta = {
            'time': 'datetime64[ns]',  # Assuming 'time' column is of datetime type
            'symbol': str,  # Assuming 'symbol' column is of string type
            'open': float,  # Assuming 'open' column is of float type
            'high': float,  # Assuming 'high' column is of float type
            'low': float,   # Assuming 'low' column is of float type
            'close': float, # Assuming 'close' column is of float type
            'volume': int,  # Assuming 'volume' column is of integer type
            'oi': int,      # Assuming 'oi' column is of integer type
            'expiry': str,  # Assuming 'expiry' column is of string type
            'strike': int,  # Assuming 'strike' column is of integer type
            'instrument_type': str  # Assuming 'instrument_type' column is of string type
        }
        # Convert delayed results to DataFrame
        df = dd.from_delayed(delayed_results, meta=meta)
    return df


# Read data from each table using Dask and concatenate them
dfs = [read_table(table_name) for table_name in relevant_table_names]

'''# Check metadata of each DataFrame
for i, df in enumerate(dfs):
    print(f"Metadata of DataFrame {i + 1}:")
    print(df.dtypes)  # Print data types of columns'''

for i, df in enumerate(dfs):
    dfs[i]['symbol'] = df['symbol'].astype('string')
    dfs[i]['expiry'] = df['expiry'].astype('string')
    dfs[i]['instrument_type'] = df['instrument_type'].astype('string')

# Concatenate DataFrames
merged_df = dd.concat(dfs)

# Print the head of the merged DataFrame
print(merged_df.head())

In [8]:
import dask.dataframe as dd
from sqlalchemy import create_engine, MetaData, Table, select
from dask.delayed import delayed

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Create an SQLAlchemy MetaData object
metadata = MetaData()

# Reflect all tables from the database
metadata.reflect(bind=engine)

# Filter relevant tables
relevant_table_names = [table_name for table_name in metadata.tables.keys() if int(table_name[-4:]) > 2017]

# Function to read each table as a Dask DataFrame
def read_table(table_name):
    # Reflect the table
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    # Define columns explicitly
    columns = [table.c.time, table.c.symbol, table.c.open, table.c.high, table.c.low, table.c.close,
               table.c.volume, table.c.oi, table.c.expiry, table.c.strike, table.c.instrument_type]
    # Create a select statement with explicit columns
    select_stmt = select(*columns).select_from(table)
    # Execute the select statement and fetch results
    with engine.connect() as conn:
        result = conn.execute(select_stmt)
        # Get metadata
        meta = {col.name: col.type.python_type for col in result.cursor.description}
        # Convert each row to a delayed DataFrame
        delayed_dfs = [dd.from_delayed([delayed(row)] * len(columns), meta=meta, columns=columns) for row in result.fetchall()]
    return delayed_dfs

# Read data from each table using Dask and concatenate them
dfs = [df for table_name in relevant_table_names for df in read_table(table_name)]
merged_df = dd.concat(dfs)

# Print the head of the merged DataFrame
print(merged_df.head())


AttributeError: 'tuple' object has no attribute 'name'

In [45]:
import dask.dataframe as dd
from sqlalchemy import create_engine

connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create SQLAlchemy engine
sql_engine = create_engine(connection_string)

tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''
import pandas as pd
tables_df = pd.read_sql_query(tables_query, sql_engine)

relevant_table_names = tables_df[tables_df['name'].str[-4:].astype(int) > 2017]['name'].tolist()

dfs = []
for table_name in relevant_table_names:
    sql_query = f"SELECT *, CAST(time AS DATETIME) AS time FROM {table_name};"
    df_dask = dd.read_sql_query(sql_query, sql_engine, index_col='time', npartitions=4)
    dfs.append(df_dask)

merged_df = dd.concat(dfs)
print(merged_df.head())


TypeError: 'con' must be of type str, not <class 'sqlalchemy.engine.base.Engine'>Note: Dask does not support SQLAlchemy connectables here

In [44]:
import dask.dataframe as dd
from sqlalchemy import create_engine

connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''
import pandas as pd
tables_df = pd.read_sql_query(tables_query, connection_string)

relevant_table_names = tables_df[tables_df['name'].str[-4:].astype(int) > 2017]['name'].tolist()

dfs = []
for table_name in relevant_table_names:
    sql_query = f"SELECT *, CAST(time AS DATETIME) AS time FROM {table_name};"
    df_dask = dd.read_sql_query(sql_query, connection_string, index_col='time', npartitions=4)
    dfs.append(df_dask)

merged_df = dd.concat(dfs)
print(merged_df.head())


AttributeError: 'str' object has no attribute 'limit'

In [ ]:
import dask.dataframe as dd
import pandas as pd
from sqlalchemy import create_engine

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create SQLAlchemy engine
sql_engine = create_engine(connection_string)

# Execute the query to get relevant table names
tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''
tables_df = pd.read_sql_query(tables_query, sql_engine)

# Filter relevant table names based on the last 4 characters (assuming they represent years)
relevant_table_names = tables_df[tables_df['name'].str[-4:].astype(int) > 2017]['name'].tolist()

# Define a list to store Dask DataFrames
dfs = []

# Loop through relevant table names
for table_name in relevant_table_names:
    # Construct SQL query to select all columns and cast 'time' as DATETIME
    sql_query = f"SELECT *, CAST(time AS DATETIME) AS time FROM `{table_name}`;"

    # Read data into a Pandas DataFrame
    df_pandas = pd.read_sql_query(sql_query, sql_engine, index_col='time')

    # Convert Pandas DataFrame to Dask DataFrame
    df_dask = dd.from_pandas(df_pandas, npartitions=4)

    # Append Dask DataFrame to the list
    dfs.append(df_dask)

# Concatenate Dask DataFrames into one
merged_df = dd.concat(dfs)

# Display the first few rows of the merged Dask DataFrame
print(merged_df.head())



In [1]:
import dask.dataframe as dd
import pandas as pd
from sqlalchemy import create_engine

# Define the connection string
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create SQLAlchemy engine
sql_engine = create_engine(connection_string)

# Execute the query to get relevant table names
tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''
tables_df = pd.read_sql_query(tables_query, sql_engine)

# Filter relevant table names based on the last 4 characters (assuming they represent years)
relevant_table_names = tables_df[tables_df['name'].str[-4:].astype(int) > 2017]['name'].tolist()

# Define a list to store Dask DataFrames
dfs = []

# Loop through relevant table names
for table_name in relevant_table_names:
    # Construct SQL query to select all columns and cast 'time' as DATETIME
    sql_query = f"SELECT *, CAST(time AS DATETIME) AS time FROM `{table_name}`;"

    # Read data into a Dask DataFrame
    df_dask = dd.read_sql_query(sql_query, sql_engine, index_col='time', npartitions=4)

    # Append Dask DataFrame to the list
    dfs.append(df_dask)

# Concatenate Dask DataFrames into one
merged_df = dd.concat(dfs)

# Display the first few rows of the merged Dask DataFrame
print(merged_df.head())


TypeError: 'con' must be of type str, not <class 'sqlalchemy.engine.base.Engine'>Note: Dask does not support SQLAlchemy connectables here

In [29]:
import dask.dataframe as dd
import pandas as pd
from sqlalchemy import text

# Define the SQL query to select data
query = text("SELECT * FROM '02012018';")

# Read the data into a Pandas DataFrame
df_pandas = pd.read_sql_query(query, connection_string)

# Check the data types of the index column
print(df_pandas.dtypes)

# Convert the 'time' column to datetime if it's not already in that format
df_pandas['time'] = pd.to_datetime(df_pandas['time'])

# Convert the Pandas DataFrame to a Dask DataFrame
df_dask = dd.from_pandas(df_pandas, npartitions=1)

# Display the first few rows of the Dask DataFrame
df_dask.head()


time                object
symbol              object
open               float64
high               float64
low                float64
close              float64
volume               int64
oi                   int64
expiry              object
strike               int64
instrument_type     object
dtype: object


<ipython-input-29-334f86d3de1a>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pandas['time'] = pd.to_datetime(df_pandas['time'])


,time,symbol,open,high,low,close,volume,oi,expiry,strike,instrument_type
0,2024-05-03 09:29:00,BANKNIFTY04JAN1823300PE,0.50,2.00,0.50,0.50,80,5800,04-01-2018,23300,PE
1,2024-05-03 09:30:00,BANKNIFTY04JAN1823300PE,2.00,2.00,2.00,2.00,80,5800,04-01-2018,23300,PE
2,2024-05-03 09:31:00,BANKNIFTY04JAN1823300PE,2.00,2.00,2.00,2.00,80,5800,04-01-2018,23300,PE
3,2024-05-03 09:34:00,BANKNIFTY04JAN1823300PE,0.15,0.15,0.15,0.15,240,5800,04-01-2018,23300,PE
4,2024-05-03 09:50:00,BANKNIFTY04JAN1823300PE,0.35,0.35,0.35,0.35,40,5800,04-01-2018,23300,PE


In [5]:
import dask.dataframe as dd
import os
from sqlalchemy import create_engine, text

# Define the connection string
connection_string = 'sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Define the SQL query to retrieve table names
tables_query = text('''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
''')

# Retrieve table names
with engine.connect() as connection:
    tables = connection.execute(tables_query)
    table_names = [table[0] for table in tables]

# Define the folder to store the CSV files
csv_folder = '/content/sample_data/Bank_Nifty_csv'
os.makedirs(csv_folder, exist_ok=True)

# Function to export each table to CSV
def export_table_to_csv(table_name):
    query = f"SELECT * FROM {table_name};"
    # Read data from the database using Dask
    df = dd.read_sql_query(query, connection_string, index_col='time')
    df = df.set_index('time')
    # Write the dataframe to a CSV file
    df.to_csv(os.path.join(csv_folder, f'{table_name}.csv'))

for table_name in table_names:
    export_table_to_csv(table_name)

csv_files = [os.path.join(csv_folder, f'{table_name}.csv') for table_name in table_names]


def read_csv(file):
    return dd.read_csv(file)

dfs = [dask.delayed(read_csv)(file) for file in csv_files]
merged_df = dd.concat(dfs)

merged_df = merged_df.compute()


AttributeError: 'str' object has no attribute 'limit'

In [ ]:
import os
import dask.dataframe as dd
from sqlalchemy import create_engine
import pandas as pd

# SQLite database path
db_path = "/content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create SQLAlchemy engine
sql_engine = create_engine(f"sqlite:///{db_path}")

# Define the SQL query to get relevant table names
tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%' AND CAST(substr(name, -4) AS INT) > 2017;
'''

# Read table names from SQLite database
relevant_table_names = pd.read_sql_query(tables_query, sql_engine)['name'].tolist()

# Define the folder to store the CSV files
csv_folder = '/content/sample_data/Bank_Nifty_csv'
os.makedirs(csv_folder, exist_ok=True)

# Function to export each table to CSV
def export_table_to_csv(table_name):
    query = f"SELECT * FROM {table_name};"
    # Read data from the database using Dask
    df = dd.read_sql_query(query, sql_engine)
    # Write the dataframe to a CSV file
    df.to_csv(os.path.join(csv_folder, f'{table_name}.csv'), index=False)

# Export each relevant table to CSV
for table_name in relevant_table_names:
    export_table_to_csv(table_name)


TypeError: read_sql_query() missing 1 required positional argument: 'index_col'

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Define the path to the SQLite database file
db_path = "/content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"

# Create a SQLAlchemy engine
engine = create_engine(f"sqlite:///{db_path}")

# Define the SQL query to select data
chunk_size = 100
relevant_table_names=relevant_table_names

# Initialize an empty list to store data chunks
all_chunks = []

# Iterate over each table in relevant_table_names
for table_name in relevant_table_names:
    query = f"SELECT * FROM `{table_name}`;"  # Enclose table name in backticks
    data_chunks = pd.read_sql_query(query, engine, chunksize=chunk_size)

    # Append each chunk to the list of chunks
    for chunk in data_chunks:
        all_chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
combined_df = pd.concat(all_chunks, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:")
display(combined_df.head())


In [21]:
import dask.dataframe as dd
from sqlalchemy import create_engine
import pandas as pd
from dask import delayed, compute

# Define the SQLAlchemy engine
engine = create_engine(connection_string)

# Create a list to store delayed tasks for reading tables
delayed_dfs = []

# Define a function to read each table and convert to Dask DataFrame
def read_table_to_dask(table_name):
    # Read the SQL table into a Pandas DataFrame
    df_pd = pd.read_sql_table(table_name, engine)

    # Convert the 'time' column to datetime with specified format
    df_pd['time'] = pd.to_datetime(df_pd['time'], format='%H:%M:%S')

    # Convert the Pandas DataFrame to a Dask DataFrame
    df_dask = dd.from_pandas(df_pd, npartitions=4)  # You can adjust the number of partitions as needed

    # Set the 'time' column as the index
    df_dask = df_dask.set_index('time')

    return df_dask

# Create delayed tasks for reading each table
for table_name in relevant_table_names:
    delayed_df = delayed(read_table_to_dask)(table_name)
    delayed_dfs.append(delayed_df)

# Compute the delayed tasks in parallel
dfs = compute(*delayed_dfs)

# Concatenate all the Dask DataFrames into a single DataFrame
merged_df = dd.concat(dfs)

# Display the first few rows of the merged DataFrame
print(merged_df.head())


KeyboardInterrupt: 

In [8]:
import dask.dataframe as dd
from sqlalchemy import create_engine

sql_engine = create_engine("sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db")
connection_string = "sqlite://///content/gdrive/MyDrive/big_file/BANKNIFTY_OPT.db"
tables_query = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%';
'''

# Connect to the database
conn = sql_engine.connect()

# Execute the query to get relevant table names
relevant_table_names =relevant_table_names
relevant_table_names = [table[0] for table in relevant_table_names]

dfs = []
for table_name in relevant_table_names:
    # Convert SQLAlchemy engine to connection string
    sql_query = f"SELECT *, CAST(time AS DATETIME) AS time FROM {table_name};"
    df_dask = dd.read_sql_query(sql_query, sql_engine, index_col='time', npartitions=4)
    dfs.append(df_dask)

merged_df = dd.concat(dfs)
print(merged_df.head())


TypeError: 'con' must be of type str, not <class 'sqlalchemy.engine.base.Engine'>Note: Dask does not support SQLAlchemy connectables here